In [1]:
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient, models
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from huggingface_hub import login
import transformers
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from deepeval.metrics import (
    ContextualPrecisionMetric,
    ContextualRecallMetric,
    ContextualRelevancyMetric,
    AnswerRelevancyMetric,
    FaithfulnessMetric,
)
from deepeval.test_case import LLMTestCase
from deepeval.models import DeepEvalBaseLLM
from deepeval import evaluate
from dotenv import load_dotenv
import os

print("Import Successful")

Import Successful


In [ ]:
load_dotenv()

In [ ]:
login(os.environ.get("HUGGINGFACE_TOKEN"))

In [ ]:
def setup_qdrant_client():
    """Initialize the Qdrant client with credentials from .env file"""
    return QdrantClient(
        url=os.environ.get("QDRANT_URL"), 
        api_key=os.environ.get("QDRANT_API_KEY"),
    )

qdrant_client = setup_qdrant_client()

In [3]:
eval_data = [
    {
        "query": "What method did AlphaGo Zero use for training?",
        "expected_answer": "AlphaGo Zero used only reinforcement learning with no human-labeled data and significantly outperformed AlphaGo.",
        "expected_contexts": [
            "AlphaGo was later succeeded by AlphaGo Zero, which was trained purely on reinforcement learning and was not given any labelled data such as human-played games to train on.",
            "AlphaGo Zero was noted to severely outperform its predecessor, winning against AlphaGo 100-0, while also requiring less computing power and training time.",
            "Considering the vast difference in performance between AlphaGo and AlphaGo Zero, it can be inferred that an AI trained only on unsupervised learning has much greater potential..."
        ]
    },
    {
        "query": "What was special about AlphaGo’s move 37 in game 2?",
        "expected_answer": "The move was highly unconventional, rated at 1 in 10,000 likelihood, and demonstrated the creativity of reinforcement learning beyond human precedent.",
        "expected_contexts": [
            "AlphaGo played an unexpectedly brilliant move so bizarre that even experts... mistakenly thought it to have been an error at first.",
            "AlphaGo calculated the probability of a human player making the same move as 1 in 10000 yet made the move anyway...",
            "This was a revelation that the AI could only have made from its simulations of games playing against itself."
        ]
    },
    {
        "query": "Why was AlphaStar’s use of disruptors notable?",
        "expected_answer": "AlphaStar adopted an advanced but rare strategy that influenced future metagame trends among professional human players.",
        "expected_contexts": [
            "AlphaStar favoured a strategy using a unit called 'disruptor'... it was uncommon at the time due to the executional difficulty of the strategy.",
            "Interestingly, the use of disruptors became more commonplace in later years...",
            "AlphaStar plays in a similar manner to professional human players but with some unique style quirks."
        ]
    },
    {
        "query": "What limitation does AlphaStar’s wall-off mistake reveal?",
        "expected_answer": "It demonstrated shallow understanding of tactical strategies, compensating with inhuman precision rather than correcting foundational mistakes.",
        "expected_contexts": [
            "AlphaStar did attempt a wall-off, it left a gap in the wall and thus left a vulnerable opening...",
            "It likely never learned that building the wall-off incorrectly is supposed to result in a disadvantaged position.",
            "As it was able to cover for the opening... it likely never recognized it as an error."
        ]
    },
    {
        "query": "What tools and languages does the candidate know?",
        "expected_answer": "The candidate is proficient in Python, R, SQL, Excel, Tableau, Power BI, Power Query, Think-Cell, and has experience with predictive modeling, ETL, and web scraping.",
        "expected_contexts": [
            "Languages & Tools: Python | R | SQL | Tableau | Excel (Advanced) | Power BI | Power Query | Think-Cell",
            "Analytics: Predictive Modeling, Statistical Inference, Optimization, Data Visualization",
            "Data Engineering: JSON/XML/HTML Parsing | Web Scraping | BeautifulSoup | ETL"
        ]
    },
    {
        "query": "What is the candidate’s experience with RAG systems?",
        "expected_answer": "Designed a RAG (Retrieval-Augmented Generation) tool for document search during a product analyst role, with top-tier team performance.",
        "expected_contexts": [
            "Product Analyst, Dubai Future Foundation... Designed an AI-powered RAG tool for intelligent document search...",
            "...ranked Top 4 among 20+ teams.",
            "Mapped UX friction points to drive feature enhancements and user engagement."
        ]
    },
    {
        "query": "What did the candidate do during their M&A project at Ferrari?",
        "expected_answer": "Conducted financial and strategic modeling for M&A at Ferrari, supporting executive decisions through data-backed insights.",
        "expected_contexts": [
            "Modeled strategic growth and synergy scenarios to support corporate development...",
            "Led cross-functional valuation and due diligence analysis (financial + strategic)...",
            "Translated quantitative findings into executive-level recommendations..."
        ]
    },
    {
        "query": "What operational risks might Marriott face in Sindalah?",
        "expected_answer": "Supply chain dependency on a new, untested port introduces operational risks, especially given the high expectations of luxury hospitality.",
        "expected_contexts": [
            "The only method of delivering any supplies to Sindalah is to go by sea.",
            "There is a high chance that they will be going through the Port of NEOM in Oxagon...",
            "...there is a significant chance of minor and major issues occurring in its early years."
        ]
    },
    {
        "query": "Why is alcohol a challenge for Marriott in NEOM?",
        "expected_answer": "Legal ambiguity and sociocultural tension around alcohol could impact guest satisfaction and local acceptance.",
        "expected_contexts": [
            "All importing, manufacturing, possession, and consumption of alcohol is illegal in Saudi Arabia for religious reasons.",
            "Claims that sales of alcohol will be allowed in Sindalah... and also claims that they will not.",
            "...Marriott faces a different challenge – how can we satisfy both the foreigners who want to consume alcohol and the locals who are against it?"
        ]
    },
    {
        "query": "Why is Sindalah a unique challenge for Marriott?",
        "expected_answer": "Sindalah’s isolation, sustainability demands, and cultural constraints make it a distinct operational and strategic challenge.",
        "expected_contexts": [
            "Marriott is not new to operating luxury hotels, yet doing so in Sindalah will still present new challenges...",
            "Sindalah is an island with no airports...",
            "Sustainability is a big topic in NEOM and thus will be expected of Sindalah and Marriott..."
        ]
    }
]


In [5]:
model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float32)
model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PhiForCausalLM(
  (model): PhiModel(
    (embed_tokens): Embedding(51200, 2560)
    (layers): ModuleList(
      (0-31): 32 x PhiDecoderLayer(
        (self_attn): PhiAttention(
          (q_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (k_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (v_proj): Linear(in_features=2560, out_features=2560, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
        )
        (mlp): PhiMLP(
          (activation_fn): NewGELUActivation()
          (fc1): Linear(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear(in_features=10240, out_features=2560, bias=True)
        )
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
      )
    )
    (rotary_emb): PhiRotaryEmbedding()
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (final_layernorm): LayerNorm((2560,), eps=1

In [6]:
def get_query_embedding(query: str):
    return embedding_model.encode(query).tolist()

def retrieve_relevant_chunks(query: str, top_k: int = 5):
    query_vector = get_query_embedding(query)

    search_result = qdrant_client.query_points(
        collection_name="ragchatbot_standard",
        query=query_vector,
        limit=5,
        with_payload=True
    )

    all_chunk_texts = []
    for _, scored_points in search_result:
        for point in scored_points:
            chunk_text = point.payload.get("chunk_text", "")
            all_chunk_texts.append(chunk_text)
    
    # Remove duplicates while preserving order
    unique_chunk_texts = list(dict.fromkeys(all_chunk_texts))
    
    return unique_chunk_texts

def build_prompt(context, question):
    return f"""You are a helpful assistant. Answer the question using only the context provided.

Context:
{context}

Question:
{question}
"""


def generate_answer(prompt, max_tokens=300):
    inputs = tokenizer(prompt, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_tokens,
            do_sample=True,
            temperature=0.7,
            top_k=50,
            top_p=0.95
        )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

embedding_model = SentenceTransformer("all-MiniLM-L6-v2")


In [7]:
class CustomLlama3_8B(DeepEvalBaseLLM):
    def __init__(self):
        quantization_config = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_quant_type="nf4",
            bnb_4bit_use_double_quant=True,
        )

        model_4bit = AutoModelForCausalLM.from_pretrained(
            "meta-llama/Meta-Llama-3-8B-Instruct",
            device_map="auto",
            quantization_config=quantization_config,
        )
        tokenizer = AutoTokenizer.from_pretrained(
            "meta-llama/Meta-Llama-3-8B-Instruct"
        )

        self.model = model_4bit
        self.tokenizer = tokenizer

    def load_model(self):
        return self.model

    def generate(self, prompt: str) -> str:
        model = self.load_model()

        pipeline = transformers.pipeline(
            "text-generation",
            model=model,
            tokenizer=self.tokenizer,
            use_cache=True,
            device_map="auto",
            max_length=2500,
            do_sample=True,
            top_k=5,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            pad_token_id=self.tokenizer.eos_token_id,
        )

        return pipeline(prompt)

    async def a_generate(self, prompt: str) -> str:
        return self.generate(prompt)

    def get_model_name(self):
        return "Llama-3 8B"

In [8]:
custom_llm = CustomLlama3_8B()

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Both `max_new_tokens` (=2048) and `max_length`(=2500) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': 'Write me a joke about a dog.\nSure, here\'s a classic joke about a dog: "Why did the dog go to the hospital? Because he had fleas!"'}]


In [9]:
test_cases = []
for item in eval_data:
    context = retrieve_relevant_chunks(item["query"])
    prompt = build_prompt(context, item["query"])
    test_cases.append(
        LLMTestCase(
            input=item["query"],
            actual_output=generate_answer(prompt),
            retrieval_context=context,
            expected_output=item["expected_answer"],
            context=item["expected_contexts"]
        )
    )

retrieval_metrics = [
    ContextualPrecisionMetric(model=custom_llm),
    ContextualRecallMetric(model=custom_llm),
    ContextualRelevancyMetric(model=custom_llm)
]

generation_metrics = [
    AnswerRelevancyMetric(model=custom_llm),
    FaithfulnessMetric(model=custom_llm)
]

print("success")

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


success


In [ ]:
evaluate(test_cases=test_cases, metrics=retrieval_metrics)

In [ ]:
evaluate(test_cases=test_cases, metrics=generation_metrics)